#Form Factor extraction
(genarated file:
"Global_ep_Crosssection_with_Poly5_Rational11\
_SigleDipole_Sum3Dipole_FromFactor.xlsx")

In [ ]:
import pandas as pd
df=pd.read_excel("resdata_protonff.xlsx.xlsx")
df

,Q2 [GeV^2],E0 [GeV],Ep [GeV],theta [deg],eps,sig [nb/sr],dsig number [nb/sr],expt.,normsys [%],First Author
0,0.4253,3.1187,2.8921,12.469,0.97000,887.700000,10.020000,27,0.015,Christy
1,0.6198,2.2380,1.9077,21.969,0.92000,109.300000,1.161000,27,0.015,Christy
2,0.6213,1.1492,0.8181,47.967,0.68000,17.050000,0.165600,27,0.015,Christy
3,0.6651,3.1187,2.7643,15.969,0.96000,187.600000,2.127000,27,0.015,Christy
4,0.8188,1.1492,0.7128,59.987,0.55000,4.804000,0.046860,27,0.015,Christy
...,...,...,...,...,...,...,...,...,...,...
448,5.0000,4.5070,1.8420,45.655,0.53826,0.008462,0.000143,1,0.018,Andivahis
449,5.0000,5.5070,2.8420,32.827,0.70417,0.021280,0.000289,1,0.018,Andivahis
450,5.0000,9.8000,7.1350,15.366,0.91902,0.157600,0.002201,1,0.018,Andivahis
451,6.0000,9.8000,6.6030,17.514,0.88628,0.047490,0.000733,1,0.018,Andivahis


In [ ]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt


In [ ]:
# --- Constants ---
M_p = 0.938272  # proton mass [GeV]
alpha = 1/137
conv = 389379   # GeV⁻²/sr to nb/sr.


In [ ]:

# --- Kinematics ---
df['theta_rad'] = np.radians(df['theta [deg]'])

df['tau'] = df['Q2 [GeV^2]'] / (4 * M_p**2)

tan_half_theta = np.tan(df['theta_rad'] / 2)
df['eps'] = 1 / (1 + 2 * (1 + df['tau'])
              * tan_half_theta**2)


# --- Mott cross-section ---
cos_half_theta = np.cos(df['theta_rad'] / 2)
sin_half_theta = np.sin(df['theta_rad'] / 2)

df['sigma_Mott'] = (
    (alpha * cos_half_theta)**2
    / (4 * df['E0 [GeV]']**2
    * sin_half_theta**4)
    * (df['Ep [GeV]'] / df['E0 [GeV]'])
    * conv
)


# --- Reduced cross-section ---
sigma_ratio = df['sig [nb/sr]'] /(
               df['sigma_Mott'])
df['sigma_R'] = df['eps'] * (1 + df['tau']) * sigma_ratio

delta_sigma_ratio = df['dsig number [nb/sr]'] / df['sigma_Mott']
df['delta_sigma_R'] = df['eps'] * (1 + df['tau']) * (
                      delta_sigma_ratio)


In [ ]:
# --- Rational (1,1) function ---
def rational_11(Q2, p0, p1, q1):
    return (p0 + p1*Q2) / (1 + q1*Q2)

# --- Rosenbluth model ---
def sigma_R_model(Q2, eps, a0, a1, b1, c0, c1, d1):
    G_M = rational_11(Q2, a0, a1, b1)
    G_E = rational_11(Q2, c0, c1, d1)
    tau = Q2 / (4 * M_p**2)
    return tau*G_M**2 + eps*G_E**2

def fit_func(xdata, a0, a1, b1, c0, c1, d1):
    Q2, eps = xdata
    return sigma_R_model(Q2, eps, a0, a1, b1, c0, c1, d1)


In [ ]:
# --- Containers ---
formfactor_results = []
radius_results = []

In [ ]:
# --- Fit per author ---
for author, subdf in df.groupby('First Author'):
    print(f"\nFitting data for author: {author}")

    Q2 = subdf['Q2 [GeV^2]'].values
    eps = subdf['eps'].values
    ydata = subdf['sigma_R'].values
    xdata = np.vstack((Q2, eps))

    p0 = [1.0, -0.1, 0.05, 1.0, -0.1, 0.05]
    bounds = ([0.5,-2,-2,0.5,-2,-2], [2,2,2,2,2,2])

    try:
        popt, _ = curve_fit(fit_func, xdata, ydata, p0=p0,
                            bounds=bounds, maxfev=200000)
        a0,a1,b1,c0,c1,d1 = popt

        # --- Fitted values ---
        G_M_fit = rational_11(Q2, a0, a1, b1)
        G_E_fit = rational_11(Q2, c0, c1, d1)
        sigma_R_fit = sigma_R_model(Q2, eps, a0,a1,b1,c0,c1,d1)

        # --- Add to formfactor results ---
        tmp_df = pd.DataFrame({
            'First Author': author,
            'Q2 [GeV^2]': Q2,
            'G_E_fit_rational_11': G_E_fit,
            'G_M_fit_rational_11': G_M_fit,
            'sigma_R_data': ydata,
            'sigma_R_fit_rational_11': sigma_R_fit,
            'Expt.': subdf['expt.'].values
        })
        formfactor_results.append(tmp_df)

        # --- Radius extraction ---
        slope_E = c1 - c0*d1
        slope_M = a1 - a0*b1
        rE2 = -6*slope_E/c0
        rM2 = -6*slope_M/a0
        rE = np.sqrt(abs(rE2))*0.1973
        rM = np.sqrt(abs(rM2))*0.1973
        radius_results.append({
            'First Author': author,
            'r_E [fm]': rE, 'r_M [fm]': rM,
            'a0':a0,'a1':a1,'b1':b1,'c0':c0,'c1':c1,'d1':d1
        })




    except RuntimeError as e:
        print(f"⚠️ Fit failed for author {author}: {e}")
        radius_results.append({'First Author': author,
                               'r_E [fm]': np.nan,
                               'r_M [fm]': np.nan,
                               'a0':np.nan,'a1':np.nan,
                               'b1':np.nan,'c0':np.nan,
                               'c1':np.nan,'d1':np.nan})
        continue



Fitting data for author: Albrecht

Fitting data for author: Andivahis

Fitting data for author: Bartel

Fitting data for author: Berger

Fitting data for author: Borkowski

Fitting data for author: Bosted

Fitting data for author: Christy

Fitting data for author: Dutta

Fitting data for author: Goiten

Fitting data for author: Janssens

Fitting data for author: Kirk

Fitting data for author: Litt


/tmp/ipython-input-3278347098.py:14: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(fit_func, xdata, ydata, p0=p0,



Fitting data for author: Murphy

Fitting data for author: Niculescu

Fitting data for author: Price

Fitting data for author: Rock

Fitting data for author: Sill

Fitting data for author: Simon

Fitting data for author: Stein

Fitting data for author: Walker


In [ ]:
# --- Combine formfactor df ---
formfactor_df = pd.concat(formfactor_results, ignore_index=True)
radius_df = pd.DataFrame(radius_results)

In [ ]:
formfactor_df

,First Author,Q2 [GeV^2],G_E_fit_rational_11,G_M_fit_rational_11,sigma_R_data,sigma_R_fit_rational_11,Expt.
0,Albrecht,4.080,0.026458,0.061753,0.004219,0.004800,18
1,Albrecht,4.160,0.023782,0.059633,0.004479,0.004509,18
2,Albrecht,4.200,0.022479,0.058600,0.004151,0.004231,18
3,Albrecht,4.880,0.003283,0.043390,0.002695,0.002615,18
4,Albrecht,5.890,-0.017687,0.026774,0.001740,0.001353,18
...,...,...,...,...,...,...,...
448,Walker,3.007,-0.048252,0.101573,0.010252,0.010261,3
449,Walker,3.007,-0.048252,0.101573,0.010524,0.010583,3
450,Walker,3.007,-0.048252,0.101573,0.010689,0.010929,3
451,Walker,3.007,-0.048252,0.101573,0.010890,0.010979,3


In [ ]:
# --- Map G_E_fit and G_M_fit_rational_11 to original df ---
G_E_map = formfactor_df.set_index(['Q2 [GeV^2]','First Author'])['G_E_fit_rational_11'].to_dict()
G_M_map = formfactor_df.set_index(['Q2 [GeV^2]','First Author'])['G_M_fit_rational_11'].to_dict()
df['G_E_fit_rational_11'] = df.apply(lambda row: G_E_map.get((row['Q2 [GeV^2]'], row['First Author'])), axis=1)
df['G_M_fit_rational_11'] = df.apply(lambda row: G_M_map.get((row['Q2 [GeV^2]'], row['First Author'])), axis=1)

# --- Preview ---
print(df[['Q2 [GeV^2]','First Author','G_E_fit_rational_11','G_M_fit_rational_11','sigma_R','delta_sigma_R']])
print("\n--- Radius Results ---")
print(radius_df)


     Q2 [GeV^2] First Author  G_E_fit_rational_11  G_M_fit_rational_11  \
0        0.4253      Christy             0.434300             0.877335   
1        0.6198      Christy             0.325107             0.694383   
2        0.6213      Christy             0.324412             0.693219   
3        0.6651      Christy             0.304910             0.660543   
4        0.8188      Christy             0.246722             0.563050   
..          ...          ...                  ...                  ...   
448      5.0000    Andivahis            -0.035370             0.032674   
449      5.0000    Andivahis            -0.035370             0.032674   
450      5.0000    Andivahis            -0.035370             0.032674   
451      6.0000    Andivahis            -0.048697             0.010899   
452      7.0000    Andivahis            -0.058471            -0.005069   

      sigma_R  delta_sigma_R  
0    0.275800       0.003113  
1    0.182976       0.001944  
2    0.160073     

In [ ]:
df

,Q2 [GeV^2],E0 [GeV],Ep [GeV],theta [deg],eps,sig [nb/sr],dsig number [nb/sr],expt.,normsys [%],First Author,theta_rad,tau,sigma_Mott,sigma_R,delta_sigma_R,G_E_fit_rational_11,G_M_fit_rational_11
0,0.4253,3.1187,2.8921,12.469,0.973946,887.700000,10.020000,27,0.015,Christy,0.217625,0.120775,3513.379405,0.275800,0.003113,0.434300,0.877335
1,0.6198,2.2380,1.9077,21.969,0.918601,109.300000,1.161000,27,0.015,Christy,0.383431,0.176009,645.303600,0.182976,0.001944,0.325107,0.694383
2,0.6213,1.1492,0.8181,47.967,0.682276,17.050000,0.165600,27,0.015,Christy,0.837182,0.176435,85.493570,0.160073,0.001555,0.324412,0.693219
3,0.6651,3.1187,2.7643,15.969,0.955310,187.600000,2.127000,27,0.015,Christy,0.278712,0.188873,1245.088627,0.171125,0.001940,0.304910,0.660543
4,0.8188,1.1492,0.7128,59.987,0.549074,4.804000,0.046860,27,0.015,Christy,1.046971,0.232520,29.257133,0.111121,0.001084,0.246722,0.563050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,5.0000,4.5070,1.8420,45.655,0.538357,0.008462,0.000143,1,0.018,Andivahis,0.796830,1.419883,3.913065,0.002817,0.000048,-0.035370,0.032674
449,5.0000,5.5070,2.8420,32.827,0.704245,0.021280,0.000289,1,0.018,Andivahis,0.572939,1.419883,12.738632,0.002847,0.000039,-0.035370,0.032674
450,5.0000,9.8000,7.1350,15.366,0.919052,0.157600,0.002201,1,0.018,Andivahis,0.268187,1.419883,120.873452,0.002900,0.000040,-0.035370,0.032674
451,6.0000,9.8000,6.6030,17.514,0.886277,0.047490,0.000733,1,0.018,Andivahis,0.305677,1.703859,66.158779,0.001720,0.000027,-0.048697,0.010899


In [ ]:
df.columns

Index(['Q2 [GeV^2]', 'E0 [GeV]', 'Ep [GeV]', 'theta [deg]', 'eps',
       'sig [nb/sr]', 'dsig number [nb/sr]', 'expt.', 'normsys [%]',
       'First Author', 'theta_rad', 'tau', 'sigma_Mott', 'sigma_R',
       'delta_sigma_R', 'G_E_fit_rational_11', 'G_M_fit_rational_11'],
      dtype='object')

#Poly5 Dipole single sum 3

In [ ]:
# --- Models ---
def poly5(Q2,p0,p1,p2,p3,p4,p5):
  return p0 + p1*Q2 + p2*Q2**2 + p3*Q2**3 + p4*Q2**4 + p5*Q2**5
def dipole(Q2,G0,Lambda2):
  return G0/(1+Q2/Lambda2)**2
def sum3_dipoles(Q2,G0_1,L1,G0_2,L2,G0_3,L3):
   return G0_1/(1+Q2/L1)**2 + G0_2/(1+Q2/L2)**2 + G0_3/(1+Q2/L3)**2

# --- Prepare columns ---
for model_type in ['poly5','dipole','sum3']:
    df[f'G_E_fit_{model_type}'] = np.nan
    df[f'G_M_fit_{model_type}'] = np.nan
    df[f'delta_G_E_{model_type}'] = np.nan
    #df[f'delta_G_E_err_{model_type}'] = np.nan
    #df[f'delta_G_M_err_{model_type}'] = np.nan

# --- Loop over models ---
for model_type in ['poly5','dipole','sum3']:
    print(f"\n===== Fitting model: {model_type} =====")

    if model_type=='poly5':
        G_model=poly5; n_params=6
    elif model_type=='dipole':
        G_model=dipole; n_params=2
    elif model_type=='sum3':
        G_model=sum3_dipoles; n_params=6

    def sigma_R_model(Q2, eps, a_params, c_params):
        G_M = G_model(Q2,*a_params)
        G_E = G_model(Q2,*c_params)
        tau = Q2/(4*M_p**2)
        return tau*G_M**2 + eps*G_E**2

    def fit_func(xdata,*params):
        Q2,eps=xdata
        a_params=params[:n_params]
        c_params=params[n_params:]
        return sigma_R_model(Q2,eps,a_params,c_params)

    for author, subdf in df.groupby('First Author'):
        Q2=subdf['Q2 [GeV^2]'].values
        eps=subdf['eps'].values
        ydata=subdf['sigma_R'].values
        xdata=np.vstack((Q2,eps))
        total_params=2*n_params
        if len(Q2)<=total_params:
            print(f"⚠️ Not enough points ({len(Q2)}) for {author} to fit {model_type} (requires > {total_params})")
            continue

        # Initial guess
        if model_type=='poly5': p0=[1.0]*12
        elif model_type=='dipole': p0=[1.0,0.71]*2
        elif model_type=='sum3': p0=[1.0,0.71,1.0,1.0,1.0,1.0]*2

        try:
            popt, pcov = curve_fit(fit_func, xdata, ydata,
                                   p0=p0, maxfev=200000)
            a_params=popt[:n_params]; c_params=popt[n_params:]
            perr=np.sqrt(np.diag(pcov))
            a_err=perr[:n_params]; c_err=perr[n_params:]

            # Fitted values
            G_M_fit=G_model(Q2,*a_params)
            G_E_fit=G_model(Q2,*c_params)
            sigma_R_fit=sigma_R_model(Q2,eps,a_params,c_params)

            # Recalculate tau for the current subdf
            tau = Q2/(4*M_p**2)

            # Map to df
            for i, q2 in enumerate(Q2):
              mask = (df['First Author'] == author) & (
                  df['Q2 [GeV^2]'] == q2)

            # Assign fitted values
              df.loc[mask, f'G_E_fit_{model_type}'] = G_E_fit[i]
              df.loc[mask, f'G_M_fit_{model_type}'] = G_M_fit[i]



        except RuntimeError as e:
            print(f"⚠️ Fit failed for {author} in {model_type}: {e}")
            continue

# --- Preview ---
cols=[col for col in df.columns if 'G_E_fit' in col or 'G_M_fit' in col]
print(df[['Q2 [GeV^2]','First Author']+cols])


===== Fitting model: poly5 =====
⚠️ Not enough points (10) for Albrecht to fit poly5 (requires > 12)
⚠️ Not enough points (11) for Bosted to fit poly5 (requires > 12)
⚠️ Not enough points (6) for Dutta to fit poly5 (requires > 12)
⚠️ Fit failed for Goiten in poly5: Optimal parameters not found: Number of calls to function has reached maxfev = 200000.
⚠️ Not enough points (7) for Kirk to fit poly5 (requires > 12)
⚠️ Fit failed for Litt in poly5: Optimal parameters not found: Number of calls to function has reached maxfev = 200000.
⚠️ Not enough points (11) for Murphy to fit poly5 (requires > 12)
⚠️ Not enough points (7) for Niculescu to fit poly5 (requires > 12)
⚠️ Not enough points (9) for Price to fit poly5 (requires > 12)
⚠️ Not enough points (5) for Rock to fit poly5 (requires > 12)
⚠️ Fit failed for Sill in poly5: Optimal parameters not found: Number of calls to function has reached maxfev = 200000.
⚠️ Not enough points (7) for Stein to fit poly5 (requires > 12)

===== Fitting mod

/tmp/ipython-input-1199430530.py:56: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, pcov = curve_fit(fit_func, xdata, ydata,
/tmp/ipython-input-1199430530.py:56: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, pcov = curve_fit(fit_func, xdata, ydata,



===== Fitting model: sum3 =====
⚠️ Not enough points (10) for Albrecht to fit sum3 (requires > 12)
⚠️ Fit failed for Bartel in sum3: Optimal parameters not found: Number of calls to function has reached maxfev = 200000.
⚠️ Not enough points (11) for Bosted to fit sum3 (requires > 12)
⚠️ Not enough points (6) for Dutta to fit sum3 (requires > 12)
⚠️ Fit failed for Goiten in sum3: Optimal parameters not found: Number of calls to function has reached maxfev = 200000.
⚠️ Not enough points (7) for Kirk to fit sum3 (requires > 12)
⚠️ Fit failed for Litt in sum3: Optimal parameters not found: Number of calls to function has reached maxfev = 200000.
⚠️ Not enough points (11) for Murphy to fit sum3 (requires > 12)
⚠️ Not enough points (7) for Niculescu to fit sum3 (requires > 12)
⚠️ Not enough points (9) for Price to fit sum3 (requires > 12)
⚠️ Not enough points (5) for Rock to fit sum3 (requires > 12)
⚠️ Fit failed for Sill in sum3: Optimal parameters not found: Number of calls to function ha

/tmp/ipython-input-1199430530.py:56: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, pcov = curve_fit(fit_func, xdata, ydata,


In [ ]:
df['delta_G_E_poly5'] = 0.5 * (
    df['G_E_fit_poly5'] * df['eps'] +
     (df['tau'] * df['G_M_fit_poly5']**2)
     / df['G_E_fit_poly5']
) * (df['delta_sigma_R'] / df['sigma_R'])

df['delta_G_E_rational_11'] = 0.5 * (
    df['G_E_fit_rational_11'] * df['eps'] +
    (df['tau'] * df['G_M_fit_rational_11']**2)
    / df['G_E_fit_rational_11']
) * (df['delta_sigma_R'] / df['sigma_R'])

df['delta_G_E_dipole'] = 0.5 * (
    df['G_E_fit_dipole'] * df['eps'] +
    (df['tau'] * df['G_M_fit_dipole']**2)
    / df['G_E_fit_dipole']
) * (df['delta_sigma_R'] / df['sigma_R'])

df['delta_G_E_sum3'] = 0.5 * (
    df['G_E_fit_sum3'] * df['eps'] +
    (df['tau'] * df['G_M_fit_sum3']**2) / df['G_E_fit_sum3']
) * (df['delta_sigma_R'] / df['sigma_R'])


In [ ]:
df.columns

Index(['Q2 [GeV^2]', 'E0 [GeV]', 'Ep [GeV]', 'theta [deg]', 'eps',
       'sig [nb/sr]', 'dsig number [nb/sr]', 'expt.', 'normsys [%]',
       'First Author', 'theta_rad', 'tau', 'sigma_Mott', 'sigma_R',
       'delta_sigma_R', 'G_E_fit_rational_11', 'G_M_fit_rational_11',
       'G_E_fit_poly5', 'G_M_fit_poly5', 'delta_G_E_poly5', 'G_E_fit_dipole',
       'G_M_fit_dipole', 'delta_G_E_dipole', 'G_E_fit_sum3', 'G_M_fit_sum3',
       'delta_G_E_sum3', 'delta_G_E_rational_11'],
      dtype='object')

In [ ]:
df.to_excel("Global_ep_Crosssection_with_Poly5_Rational11_SigleDipole_Sum3Dipole_FromFactor.xlsx")